In [32]:
%pip install pandas scikit-learn torch gradio

In [26]:
# Load the dataset
df = pd.read_csv('/content/healthcare_dataset.csv')

# Display the first few rows
display(df.head())

# Check for missing values
display(df.isnull().sum())

# Create the 'Healthy' target variable
df['Healthy'] = df['Test Results'].apply(lambda x: 1 if x == 'Normal' else 0)

# Convert categorical features to numerical using Label Encoding
categorical_cols = ['Gender', 'Blood Type', 'Medical Condition', 'Insurance Provider', 'Admission Type', 'Medication', 'Test Results']
for col in categorical_cols:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])

# Drop columns that are not suitable for training or have too many unique values
columns_to_drop = ['Name', 'Date of Admission', 'Doctor', 'Hospital', 'Room Number', 'Discharge Date', 'Test Results']
existing_columns_to_drop = [col for col in columns_to_drop if col in df.columns]
df = df.drop(existing_columns_to_drop, axis=1)

# Define features (X) and target (y)
X = df.drop('Healthy', axis=1)
y = df['Healthy']

# Display the first few rows of the preprocessed data
display(X.head())
display(y.head())

,Name,Age,Gender,Blood Type,Medical Condition,Date of Admission,Doctor,Hospital,Insurance Provider,Billing Amount,Room Number,Admission Type,Discharge Date,Medication,Test Results
0,Bobby JacksOn,30,Male,B-,Cancer,2024-01-31,Matthew Smith,Sons and Miller,Blue Cross,18856.281306,328,Urgent,2024-02-02,Paracetamol,Normal
1,LesLie TErRy,62,Male,A+,Obesity,2019-08-20,Samantha Davies,Kim Inc,Medicare,33643.327287,265,Emergency,2019-08-26,Ibuprofen,Inconclusive
2,DaNnY sMitH,76,Female,A-,Obesity,2022-09-22,Tiffany Mitchell,Cook PLC,Aetna,27955.096079,205,Emergency,2022-10-07,Aspirin,Normal
3,andrEw waTtS,28,Female,O+,Diabetes,2020-11-18,Kevin Wells,"Hernandez Rogers and Vang,",Medicare,37909.782410,450,Elective,2020-12-18,Ibuprofen,Abnormal
4,adrIENNE bEll,43,Female,AB+,Cancer,2022-09-19,Kathleen Hanna,White-White,Aetna,14238.317814,458,Urgent,2022-10-09,Penicillin,Abnormal


,0
Name,0
Age,0
Gender,0
Blood Type,0
Medical Condition,0
Date of Admission,0
Doctor,0
Hospital,0
Insurance Provider,0
Billing Amount,0


,Age,Gender,Blood Type,Medical Condition,Insurance Provider,Billing Amount,Admission Type,Medication
0,30,1,5,2,1,18856.281306,2,3
1,62,1,0,5,3,33643.327287,1,1
2,76,0,1,5,0,27955.096079,1,0
3,28,0,6,3,3,37909.782410,0,1
4,43,0,2,2,0,14238.317814,2,4


,Healthy
0,1
1,0
2,1
3,0
4,0


## Define and train a model

### Subtask:
Define a classification model using PyTorch and train it on the preprocessed data to predict the 'Healthy' status.


**Reasoning**:
Define a simple neural network for binary classification using PyTorch, split the data, scale the features, and train the model.



In [27]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Convert to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32).reshape(-1, 1)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32).reshape(-1, 1)

# Define the classification model
class HealthcareClassifier(nn.Module):
    def __init__(self, input_dim):
        super(HealthcareClassifier, self).__init__()
        self.fc1 = nn.Linear(input_dim, 64)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(64, 32)
        self.relu = nn.ReLU()
        self.fc3 = nn.Linear(32, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc3(x)
        x = self.sigmoid(x)
        return x

# Initialize the model, loss function, and optimizer
input_dim = X_train_tensor.shape[1]
model = HealthcareClassifier(input_dim)
criterion = nn.BCEWithLogitsLoss() # Using BCEWithLogitsLoss for binary classification
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train the model
epochs = 100
for epoch in range(epochs):
    optimizer.zero_grad()
    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)
    loss.backward()
    optimizer.step()

    if (epoch+1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}')

print("Training finished.")

Epoch [10/100], Loss: 0.7844
Epoch [20/100], Loss: 0.7628
Epoch [30/100], Loss: 0.7403
Epoch [40/100], Loss: 0.7211
Epoch [50/100], Loss: 0.7086
Epoch [60/100], Loss: 0.7018
Epoch [70/100], Loss: 0.6983
Epoch [80/100], Loss: 0.6965
Epoch [90/100], Loss: 0.6955
Epoch [100/100], Loss: 0.6949
Training finished.


## Evaluate the model

### Subtask:
Evaluate the trained model's performance using appropriate classification metrics.


**Reasoning**:
Use the trained PyTorch model to make predictions on the test set, convert predictions to class labels, and evaluate the model's performance using classification metrics.



In [28]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Make predictions on the test set
with torch.no_grad():
    y_pred_tensor = model(X_test_tensor)

# Convert predictions to class labels (0 or 1)
y_pred_labels = (y_pred_tensor > 0.5).int()

# Convert tensors to NumPy arrays
y_test_np = y_test_tensor.numpy()
y_pred_np = y_pred_labels.numpy()

# Calculate evaluation metrics
accuracy = accuracy_score(y_test_np, y_pred_np)
precision = precision_score(y_test_np, y_pred_np)
recall = recall_score(y_test_np, y_pred_np)
f1 = f1_score(y_test_np, y_pred_np)

# Print the metrics
print(f'Accuracy: {accuracy:.4f}')
print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'F1-score: {f1:.4f}')

Accuracy: 0.6641
Precision: 0.0000
Recall: 0.0000
F1-score: 0.0000


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


**Reasoning**:
The previous evaluation metrics show that the model is not predicting any positive cases (Healthy), resulting in zero precision, recall, and F1-score. This could be due to the threshold of 0.5 or the model not being trained well enough for the classification task. I will try lowering the threshold for converting probabilities to class labels to see if it improves the metrics.



In [29]:
# Convert predictions to class labels with a lower threshold (e.g., 0.3)
y_pred_labels_lower_threshold = (y_pred_tensor > 0.3).int()

# Convert tensors to NumPy arrays
y_test_np = y_test_tensor.numpy()
y_pred_np_lower_threshold = y_pred_labels_lower_threshold.numpy()

# Calculate evaluation metrics with the lower threshold
accuracy_lt = accuracy_score(y_test_np, y_pred_np_lower_threshold)
precision_lt = precision_score(y_test_np, y_pred_np_lower_threshold)
recall_lt = recall_score(y_test_np, y_pred_np_lower_threshold)
f1_lt = f1_score(y_test_np, y_pred_np_lower_threshold)

# Print the metrics with the lower threshold
print(f'Metrics with threshold 0.3:')
print(f'Accuracy: {accuracy_lt:.4f}')
print(f'Precision: {precision_lt:.4f}')
print(f'Recall: {recall_lt:.4f}')
print(f'F1-score: {f1_lt:.4f}')

Metrics with threshold 0.3:
Accuracy: 0.6641
Precision: 0.0000
Recall: 0.0000
F1-score: 0.0000


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


## Create a gradio interface

### Subtask:
Build a Gradio interface that takes the relevant features as input and outputs the predicted 'Healthy' status.


**Reasoning**:
Define the prediction function for the Gradio interface, which will take the input features, scale them, make a prediction using the trained PyTorch model, and return the predicted class label (0 or 1). Then, create the Gradio interface with appropriate input components for each feature and a text output component.



In [30]:
import gradio as gr
import numpy as np

# Define the prediction function
def predict_healthy_status(Age, Gender, Blood_Type, Medical_Condition, Insurance_Provider, Admission_Type, Medication):
    # Create a numpy array from the input values
    input_data = np.array([[Age, Gender, Blood_Type, Medical_Condition, Insurance_Provider, Admission_Type, Medication]])

    # Scale the input data using the same scaler used during training
    input_data_scaled = scaler.transform(input_data)

    # Convert the scaled input data to a PyTorch tensor
    input_tensor = torch.tensor(input_data_scaled, dtype=torch.float32)

    # Get the prediction from the model
    with torch.no_grad():
        prediction = model(input_tensor)

    # Convert the model's output (probability) to a predicted class label (0 or 1)
    predicted_class = 1 if prediction.item() > 0.5 else 0

    # Return the predicted class label
    return "Healthy" if predicted_class == 1 else "Not Healthy"


# Create the Gradio interface
interface = gr.Interface(
    fn=predict_healthy_status,
    inputs=[
        gr.Number(label="Age"),
        gr.Number(label="Gender (0: Female, 1: Male)"),
        gr.Number(label="Blood Type (Encoded: 0-7)"),
        gr.Number(label="Medical Condition (Encoded: 0-7)"),
        gr.Number(label="Insurance Provider (Encoded: 0-4)"),
        gr.Number(label="Admission Type (Encoded: 0-2)"),
        gr.Number(label="Medication (Encoded: 0-4)"),
    ],
    outputs=gr.Textbox(label="Predicted Health Status")
)

**Reasoning**:
Launch the Gradio interface that was created in the previous step.



In [31]:
interface.launch()

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://3849b0a59f7667f329.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


## Summary:

### Data Analysis Key Findings

*   The dataset was loaded successfully with no missing values.
*   A new target variable 'Healthy' was created, where 'Normal' test results are considered Healthy (1) and others are Not Healthy (0).
*   Categorical features were converted to numerical representations using Label Encoding.
*   Irrelevant columns such as 'Name', 'Date of Admission', and 'Doctor' were dropped.
*   The data was split into training (80%) and testing (20%) sets, and features were scaled using `StandardScaler`.
*   A PyTorch binary classification model was defined and trained for 100 epochs.
*   Model evaluation using the default threshold (0.5) resulted in an accuracy of 0.6641 but a precision, recall, and F1-score of 0.0000, indicating the model did not predict any positive cases ('Healthy').
*   Lowering the classification threshold to 0.3 did not improve the precision, recall, or F1-score, which remained at 0.0000.
*   A Gradio interface was successfully created and launched to allow users to input healthcare features and receive a predicted health status ('Healthy' or 'Not Healthy') based on the trained model.

### Insights or Next Steps

*   The current model exhibits poor performance in identifying 'Healthy' individuals. The evaluation metrics suggest the model is biased towards predicting the negative class.
*   Further steps should involve investigating the class distribution in the dataset (imbalanced classes might be an issue), exploring different model architectures, adjusting hyperparameters, or considering techniques like oversampling or undersampling to improve the model's ability to predict the positive class.
